In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
#read the csv file into a dataframe
real_state_data = pd.read_csv("realtordata.csv")
income = pd.read_csv("Income.csv")
population = pd.read_csv("population.csv")
land = pd.read_csv("land.csv")
st_abr = pd.read_csv("state_abr.csv")
geo_data= pd.read_csv("geo-data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'realtordata.csv'

In [ ]:
real_state_data = pd.read_csv("realtordata.csv")
geo_data= pd.read_csv("geo-data.csv")

In [ ]:
real_state_data.head()

In [ ]:
real_state_data=pd.DataFrame (real_state_data , columns =['price','zip_code', 'bed', 'bath', 'acre_lot','house_size'])
real_state_data

In [ ]:
real_state_data=real_state_data.dropna(axis=0)
real_state_data.isnull().sum()


In [ ]:
real_state_data['zip_code'] = real_state_data['zip_code'].astype(int)
real_state_data['zip_code'] = real_state_data['zip_code'].astype(str)
geo_data['zipcode']=geo_data['zipcode'].astype(str)

In [ ]:
real_state_data['zip_code']=real_state_data['zip_code'].apply(lambda x: x.zfill(5))
real_state_data.head()

In [ ]:
df1 = pd.DataFrame (real_state_data , columns = ['zip_code', 'price', 'house_size' ])
realstate_grouped = df1.groupby(['zip_code'],as_index=False).mean()
realstate_grouped['price_per_sqft']=realstate_grouped['price']/realstate_grouped['house_size']
realstate_grouped.head()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

X = realstate_grouped[['price_per_sqft']].values

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

n_clusters=3
kmeans = KMeans(n_clusters=n_clusters, random_state=10)

kmeans.fit(X_std)

labels = kmeans.predict(X_std)

for n, i in enumerate(labels):
    if i==0:
        realstate_grouped.loc[n,'Cluster']=i+1
    elif i==1:
        realstate_grouped.loc[n,'Cluster']=i+2
    elif i==2:
        realstate_grouped.loc[n,'Cluster']=i


    

# Visualize clusters
colors = ['red', 'green', 'blue']
plt.scatter(realstate_grouped.index,X_std[:,0], c=labels)
plt.ylabel('Average House Price (standardized)')
plt.show()

plt.hist(X[labels==0], label='Cluster 1')
plt.hist(X[labels==2], label='Cluster 2')
plt.hist(X[labels==1],label='Cluster 3')
plt.xlabel('Average House Price')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
 realstate_grouped.head()

In [ ]:
realstate_grouped=realstate_grouped.loc[:,['zip_code','Cluster']]
realstate_clustered=real_state_data.merge(realstate_grouped, how='inner', on=['zip_code'])
realstate_clustered.head()

In [ ]:
realstate_clustered.rename(columns = {'zip_code':'zipcode'}, inplace = True)
real_state_bycounty=realstate_clustered.merge(geo_data, how='inner', on='zipcode')
real_state=real_state_bycounty.drop(['state_fips','state','city'],axis=1)
real_state.rename(columns = {'state_abbr':'State'}, inplace = True)
real_state.head()

In [ ]:
real_state.rename(columns = {'county':'County'}, inplace = True)

In [ ]:
income.isnull().sum()

In [ ]:
income=pd.DataFrame (income , columns =['State','County','PerCapitaInc'])
income=income.dropna(axis=0)
income


In [ ]:
realstate_income=real_state.merge(income, how='inner', on=['State','County'])
realstate_income.head()

In [ ]:
df3 = pd.DataFrame (land , columns = ['USPS','NAME', 'ALAND' ])
df3.rename(columns = {'USPS':'State'}, inplace = True)
df3.rename(columns = {'NAME':'County'}, inplace = True)
df3


In [ ]:
i=0
for c in df3.County:
    Cnt=c.split(' County')[0]
    df3.loc[i,'County']=Cnt
    i+=1
df3

In [ ]:
realstate_income_land=realstate_income.merge(df3, how='inner', on=['State','County'])
realstate_income_land

In [ ]:
df4 = pd.DataFrame (population , columns = ['State', 'CountyName', 'TotalPop' ])
df4


In [ ]:
df4.isnull().sum()

In [ ]:
df4=df4.merge(st_abr, how='inner', on='State')
df4

In [ ]:
i=0
for c in df4.CountyName:
    Cnt=c.split(' County')[0]
    df4.loc[i,'CountyName']=Cnt
    i+=1
df4

In [ ]:
population=df4.drop(['State'], axis=1)
population.rename(columns = {'CountyName':'County'}, inplace = True)
population.rename(columns = {'Postal':'State'}, inplace = True)
population

In [ ]:
realstate_income_land_population=realstate_income_land.merge(population, how='inner', on=['State', 'County'])
realstate_income_land_population['Price_perft2']=realstate_income_land_population['price']/realstate_income_land_population['house_size']
realstate_income_land_population

In [ ]:
from scipy import stats
dfnn=realstate_income_land_population.copy()
def remove_outliers_zscore(df_column):
    z_scores = stats.zscore(df_column)
    abs_z_scores = abs(z_scores)
    filtered_entries = (abs_z_scores < 3)
    return df_column[filtered_entries]

realstate_income_land_population[['bed', 'bath','acre_lot','house_size','Price_perft2']] = realstate_income_land_population[['bed', 'bath','acre_lot','house_size','Price_perft2']].apply(remove_outliers_zscore)

realstate_income_land_population=realstate_income_land_population.dropna()

In [ ]:
import seaborn as sns

correlation = realstate_income_land_population.drop(columns=['Cluster','Price_perft2']).corr()
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(correlation,annot=True)

In [ ]:
abs(correlation['price']).sort_values(ascending=False)

In [ ]:
realstate_income_land_population['size_bin'] = pd.cut(realstate_income_land_population['house_size'], bins=10)

grouped_data = realstate_income_land_population.groupby(['Cluster', 'size_bin'])['price'].mean().reset_index()

pivot_data = grouped_data.pivot(index='size_bin', columns='Cluster', values='price')


ax=pivot_data.plot(kind='bar')
plt.xlabel('House Size Bins')
plt.ylabel('Average Price')
ax.legend(labels=['Cluster 1', 'Cluster 2', 'Cluster 3'])
plt.show()

In [ ]:
grouped = realstate_income_land_population.groupby(['Cluster','bath']).mean().reset_index()
fig, ax = plt.subplots()
colors = ['red', 'green', 'blue']
for i, c in enumerate([1, 2, 3]):
    subset = grouped[grouped['Cluster'] == c]
    ax.scatter(subset['bath'], subset['price'], color=colors[i], label=f'Cluster {c}')
    plt.xlabel("Number of Baths")
    plt.ylabel("Mean Price")
ax.legend()
plt.show()

In [ ]:
grouped = realstate_income_land_population.groupby(['Cluster','bed']).mean().reset_index()
fig, ax = plt.subplots()
colors = ['red', 'green', 'blue']
for i, c in enumerate([1, 2, 3]):
    subset = grouped[grouped['Cluster'] == c]
    ax.scatter(subset['bed'], subset['price'], color=colors[i], label=f'Cluster {c}')
    plt.xlabel("Number of Bedrooms")
    plt.ylabel("Mean Price")
ax.legend()
plt.show()



In [ ]:
realstate_income_land_population['acre_bin'] = pd.cut(realstate_income_land_population['acre_lot'], bins=10)

grouped_data = realstate_income_land_population.groupby(['Cluster', 'acre_bin'])['price'].mean().reset_index()

pivot_data = grouped_data.pivot(index='acre_bin', columns='Cluster', values='price')


ax=pivot_data.plot(kind='bar')
plt.xlabel('Acre Bins')
plt.ylabel('Average Price')
ax.legend(labels=['Cluster 1', 'Cluster 2', 'Cluster 3'])
plt.yscale("log")
plt.show()

In [ ]:
Final_data=pd.DataFrame (realstate_income_land_population , columns =['price', 'bed', 'bath', 'acre_lot','house_size', 'Cluster', 'PerCapitaInc', 'ALAND', 'TotalPop'])
Final_data.head()

In [34]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

scaler = StandardScaler()


# number of folds
k = 4

kf = KFold(n_splits=k, shuffle=True, random_state=10)

r2_linreg_test_list = []
r2_rf_test_list = []
#r2_svm_test_list = []

for cluster in range(1, n_clusters+1):
    
    
    X_cluster = Final_data[Final_data['Cluster'] == cluster].drop(['Cluster','price'], axis=1)
    y_cluster = Final_data[Final_data['Cluster'] == cluster]['price']
    
    X_train_val, X_test, y_train_val, y_test = train_test_split(X_cluster, y_cluster, test_size=0.2, random_state=10)
    
    # Evaluation metrics
    r2_linreg_list = []
    r2_rf_list = []
    #r2_svm_list = []
    
    # Loop through each fold
    for train_idx, val_idx in kf.split(X_train_val):
        
        # Split the data into training and validation sets
        X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
        y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
        
        # Scale the features 
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        # Fit a linear regression model to the data
        linreg_model = LinearRegression()
        linreg_model.fit(X_train_scaled, y_train)
        y_linreg_pred = linreg_model.predict(X_val_scaled)
        
        # Fit a random forest model to the data
        rf_model = RandomForestRegressor(n_estimators=100, random_state=10)
        rf_model.fit(X_train_scaled, y_train)
        y_rf_pred = rf_model.predict(X_val_scaled)
        
        # Fit a SVM model to the data
        #svm_model = SVR(kernel='linear')
        #svm_model.fit(X_train_scaled, y_train)
        #y_svm_pred = svm_model.predict(X_val_scaled)
        
        # Calculate R2 for each model and append to the respective list
        r2_linreg_list.append(r2_score(y_val, y_linreg_pred))
        r2_rf_list.append(r2_score(y_val, y_rf_pred))
        #r2_svm_list.append(r2_score(y_val, y_svm_pred))
    
    #Prediction of the test set
    X_test_scaled = scaler.transform(X_test)
    
    y_linreg_test_pred = linreg_model.predict(X_test_scaled)
    r2_linreg_test_list.append(r2_score(y_test, y_linreg_test_pred))
    
    y_rf_test_pred = rf_model.predict(X_test_scaled)
    r2_rf_test_list.append(r2_score(y_test, y_rf_test_pred))
    
    #y_svm_test_pred = svm_model.predict(X_test_scaled)
    #r2_svm_test_list.append(r2_score(y_test, y_svm_test_pred))
    
    
    # Print the mean R2 for validation set
    print(f"Cluster {cluster}:")
    print(f"   Validation Mean R2 Linear Regression = {sum(r2_linreg_list)/k:.3f}")
    print(f"   Validation Mean R2 Random Forest = {sum(r2_rf_list)/k:.3f}")
    #print(f"  Validation Mean R2 SVM = {sum(r2_svm_list)/k:.3f}")
    
    
# Print the mean R2 for test set
print('\n')
print(f"Test Mean R2 Linear Regression = {sum(r2_linreg_test_list)/n_clusters:.3f}")
print(f"Test Mean R2 Random Forest = {sum(r2_rf_test_list)/n_clusters:.3f}")
#print(f"Test Mean R2 SVM = {sum(r2_svm_test_list)/n_clusters:.3f}")

Cluster 1:
   Validation Mean R2 Linear Regression = 0.546
   Validation Mean R2 Random Forest = 0.982
Cluster 2:
   Validation Mean R2 Linear Regression = 0.671
   Validation Mean R2 Random Forest = 0.981
Cluster 3:
   Validation Mean R2 Linear Regression = 0.890
   Validation Mean R2 Random Forest = 0.983


Test Mean R2 Linear Regression = 0.697
Test Mean R2 Random Forest = 0.982


In [35]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

scaler = StandardScaler()

# number of folds
k = 4

kf = KFold(n_splits=k, shuffle=True, random_state=10)

r2_linreg_test_list = []
r2_rf_test_list = []
#r2_svm_test_list = []


for cluster in range(1, n_clusters+1):
    
    
    X_cluster = Final_data[Final_data['Cluster'] == cluster].drop(['Cluster','price'], axis=1)
    y_cluster = Final_data[Final_data['Cluster'] == cluster]['price']
    
    X_train_val, X_test, y_train_val, y_test = train_test_split(X_cluster, y_cluster, test_size=0.2, random_state=10)
    
    X_train_val_scaled = scaler.fit_transform(X_train_val)
    
    #Use PCA to reduce the number of features
    pca = PCA()
    X_train_val_pca = pca.fit_transform(X_train_val_scaled)
    
    variance_ratio = pca.explained_variance_ratio_
    n_components = 0
    explained_variance = 0
    for i in range(len(variance_ratio)):
        explained_variance += variance_ratio[i]
        n_components += 1
        if explained_variance >= 0.9:
            break
    print(f"Cluster {cluster}: {n_components} principal components explain 90% of the variance")
    
    pca = PCA(n_components=n_components)
    X_train_val_pca = pca.fit_transform(X_train_val)
    X_test_pca = pca.transform(X_test)
    
    
    # Evaluation metrics
    r2_linreg_list = []
    r2_rf_list = []
    #r2_svm_list = []
    
    # Loop through each fold
    for train_idx, val_idx in kf.split(X_train_val_pca):
        
        # Split the data into training and validation sets
        X_train, X_val = X_train_val_pca[train_idx], X_train_val_pca[val_idx]
        y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
        
        # Scale the features 
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        # Fit a linear regression model to the data
        linreg_model = LinearRegression()
        linreg_model.fit(X_train_scaled, y_train)
        y_linreg_pred = linreg_model.predict(X_val_scaled)
        
        # Fit a random forest model to the data
        rf_model = RandomForestRegressor(n_estimators=100, random_state=10)
        rf_model.fit(X_train_scaled, y_train)
        y_rf_pred = rf_model.predict(X_val_scaled)
        
        # Fit a SVM model to the data
        #svm_model = SVR(kernel='linear')
        #svm_model.fit(X_train_scaled, y_train)
        #y_svm_pred = svm_model.predict(X_val_scaled)
        
        # Calculate R2 for each model and append to the respective list
        r2_linreg_list.append(r2_score(y_val, y_linreg_pred))
        r2_rf_list.append(r2_score(y_val, y_rf_pred))
        #r2_svm_list.append(r2_score(y_val, y_svm_pred))
    
    #Prediction of the test set
    X_test_scaled = scaler.transform(X_test_pca)
    
    y_linreg_test_pred = linreg_model.predict(X_test_scaled)
    r2_linreg_test_list.append(r2_score(y_test, y_linreg_test_pred))
    
    y_rf_test_pred = rf_model.predict(X_test_scaled)
    r2_rf_test_list.append(r2_score(y_test, y_rf_test_pred))
    
    #y_svm_test_pred = svm_model.predict(X_test_scaled)
    #r2_svm_test_list.append(r2_score(y_test, y_svm_test_pred))
    
    
    # Print the mean R2 for validation set
    print(f"   Validation Mean R2 Linear Regression = {sum(r2_linreg_list)/k:.3f}")
    print(f"   Validation Mean R2 Random Forest = {sum(r2_rf_list)/k:.3f}")
    #print(f"  Validation Mean R2 SVM = {sum(r2_svm_list)/k:.3f}")
    
    
# Print the mean R2 for test set
print('\n')
print(f"Test Mean R2 Linear Regression = {sum(r2_linreg_test_list)/n_clusters:.3f}")
print(f"Test Mean R2 Random Forest = {sum(r2_rf_test_list)/n_clusters:.3f}")
#print(f"Test Mean R2 SVM = {sum(r2_svm_test_list)/n_clusters:.3f}")


Cluster 1: 5 principal components explain 90% of the variance
   Validation Mean R2 Linear Regression = 0.497
   Validation Mean R2 Random Forest = 0.978
Cluster 2: 5 principal components explain 90% of the variance
   Validation Mean R2 Linear Regression = 0.655
   Validation Mean R2 Random Forest = 0.979
Cluster 3: 4 principal components explain 90% of the variance
   Validation Mean R2 Linear Regression = 0.875
   Validation Mean R2 Random Forest = 0.956


Test Mean R2 Linear Regression = 0.671
Test Mean R2 Random Forest = 0.949
